In [1]:
import pandas as pd
import numpy as np

In [2]:
rootDir=r'C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed Data'
TrainDtCsvDir=rootDir+r'\Train_RevisionWithOriTbreak.csv'
ValDtCsvDir=rootDir+r'Val_Tbreak_Yield.csv'
ValDtCsvDir0=rootDir+r'\Val_Tbreak_Yield_0Added.csv'
TrainDtFxwithPP=rootDir+r'\FixedTbreakWithPP.csv'
TrainExtendDir=r'C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\Tbreak_Yield.csv'

In [3]:
trainDtDf=pd.read_csv(TrainDtCsvDir)
trainDtDfFixPP=pd.read_csv(TrainDtFxwithPP)
TrainExtend=pd.read_csv(TrainExtendDir)

In [4]:
"""
    This function calculates the modified value of Tbreak based on the yield at Tbreak
    and assigns a point system that satisfies various criteria related to temperature 
    and yield. The function is based on the following rule-based system:

    1. **Tbreak < 673K (Level 1)**:
        - If the temperature is below 673K, the system returns the negative of yield_at_Tbreak.
        - No criteria points are added.
    
    2. **673K ≤ Tbreak < 873K (Level 2)**:
        - If yield_at_Tbreak > 1200:
            - 3 points are assigned (2 points for satisfying the temp range, 1 additional for high yield).
            - Tbreak is updated by adding yield_at_Tbreak * 1.5.
        - If yield_at_Tbreak > 900:
            - 2 points are assigned (satisfies both temp range and yield conditions).
            - Tbreak is updated by adding yield_at_Tbreak.
        - If yield_at_Tbreak <= 900:
            - 1 point is assigned (only the temp range condition is satisfied).
            - Tbreak is returned as is.

    3. **873K ≤ Tbreak < 1073K (Level 3)**:
        - If yield_at_Tbreak > 900:
            - 4 points are assigned (3 for temp and yield, 1 extra for high yield).
            - Tbreak is updated by adding yield_at_Tbreak * 4.
        - If yield_at_Tbreak > 700:
            - 3 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 2.
        - If yield_at_Tbreak > 500:
            - 2 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 1.5.
        - If yield_at_Tbreak <= 500:
            - 1 point is assigned (only temp range is satisfied).
            - Half of yield_at_Tbreak is returned.

    4. **1073K ≤ Tbreak < 1274K (Level 4)**:
        - If yield_at_Tbreak > 700:
            - 4 points are assigned (3 for temp and yield, 1 extra for high yield).
            - Tbreak is updated by adding yield_at_Tbreak * 6.
        - If yield_at_Tbreak > 500:
            - 3 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 4.
        - If yield_at_Tbreak > 300:
            - 2 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 2.
        - If yield_at_Tbreak <= 300:
            - 1 point is assigned (only temp range is satisfied).
            - Half of yield_at_Tbreak is returned.

    5. **Tbreak ≥ 1274K (Level 5)**:
        - If yield_at_Tbreak > 500:
            - 4 points are assigned (3 for temp and yield, 1 extra for high yield).
            - Tbreak is updated by adding yield_at_Tbreak * 9.
        - If yield_at_Tbreak > 300:
            - 3 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 6.
        - If yield_at_Tbreak > 150:
            - 2 points are assigned (satisfies temp range and yield condition).
            - Tbreak is updated by adding yield_at_Tbreak * 3.
        - If yield_at_Tbreak <= 150:
            - 1 point is assigned (only temp range is satisfied).
            - Half of yield_at_Tbreak is returned.
    
    The function returns the modified value of Tbreak, the number of criteria satisfied, 
    and the temperature range level.
    """
def process_Tbreak(Tbreak, yield_at_Tbreak):

    criteria_satisfied = 0  # Initialize the counter for satisfied criteria
    temp_range_level = None  # Initialize variable for temperature range level

    # Rule for Tbreak < 673K (Level 1)
    if Tbreak < 673:
        temp_range_level = 1
        return -yield_at_Tbreak, criteria_satisfied, temp_range_level

    # Rule for Tbreak between 673K and 873K (Level 2)
    if 673 <= Tbreak < 873:
        temp_range_level = 2
        if yield_at_Tbreak > 1200:
            criteria_satisfied += 3  # Satisfies temp range and highest yield condition (2 + 1 extra)
            return Tbreak + yield_at_Tbreak * 1.5, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 900:
            criteria_satisfied += 2  # Satisfies the temperature range and yield condition
            return Tbreak + yield_at_Tbreak, criteria_satisfied, temp_range_level
        else:
            criteria_satisfied += 1  # Satisfies only the temperature range condition
            return Tbreak, criteria_satisfied, temp_range_level

    # Rule for Tbreak between 873K and 1073K (Level 3)
    if 873 <= Tbreak < 1073:
        temp_range_level = 3
        if yield_at_Tbreak > 900:
            criteria_satisfied += 4  # Satisfies temp range and highest yield condition (2 + 1 extra)
            return Tbreak + yield_at_Tbreak * 4, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 700:
            criteria_satisfied += 3  # Satisfies the temperature range and yield condition
            return Tbreak + yield_at_Tbreak * 2, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 500:
            criteria_satisfied += 2  # Satisfies the temperature range and yield condition
            return Tbreak + yield_at_Tbreak * 1.5, criteria_satisfied, temp_range_level
        else:
            criteria_satisfied += 1  # Satisfies only the temperature range condition
            return yield_at_Tbreak/2, criteria_satisfied, temp_range_level

    # Rule for Tbreak between 1073K and 1274K (Level 4)
    if 1073 <= Tbreak < 1274:
        temp_range_level = 4
        if yield_at_Tbreak > 700:
            criteria_satisfied += 4  # Satisfies temp range and highest yield condition (2 + 1 extra)
            return Tbreak + yield_at_Tbreak * 6, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 500:
            criteria_satisfied += 3  # Satisfies temp range and yield condition
            return Tbreak + yield_at_Tbreak * 4, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 300:
            criteria_satisfied += 2  # Satisfies temp range and yield condition
            return Tbreak + yield_at_Tbreak * 2, criteria_satisfied, temp_range_level
        else:
            criteria_satisfied += 1  # Satisfies only the temperature range condition
            return yield_at_Tbreak/2, criteria_satisfied, temp_range_level

    # Rule for Tbreak > 1273K (Level 5)
    if Tbreak >= 1274:
        temp_range_level = 5
        if yield_at_Tbreak > 500:
            criteria_satisfied += 4  # Satisfies temp range and highest yield condition (2 + 1 extra)
            return Tbreak + yield_at_Tbreak * 9, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 300:
            criteria_satisfied += 3  # Satisfies temp range and yield condition
            return Tbreak + yield_at_Tbreak * 6, criteria_satisfied, temp_range_level
        elif yield_at_Tbreak > 150:
            criteria_satisfied += 2  # Satisfies temp range and yield condition
            return Tbreak + yield_at_Tbreak * 3, criteria_satisfied, temp_range_level
        else:
            criteria_satisfied += 1  # Satisfies only the temperature range condition
            return yield_at_Tbreak/2, criteria_satisfied, temp_range_level

    # Default return (should not be reached based on the conditions)
    return None, criteria_satisfied, temp_range_level


In [14]:
pts=[]
criteria_satisfieds=[]
temp_range_levels=[]
for tbreak,tyield in zip(trainDtDf['Tbreak (K)'],trainDtDf['YS at Tbreak']):
    pt,criteria_satisfied, temp_range_level=process_Tbreak(tbreak,tyield)
    pts.append(pt)
    criteria_satisfieds.append(criteria_satisfied)
    temp_range_levels.append(temp_range_level)
trainDtDf['Assigned Point']=pts
trainDtDf['Criteria Satisfied']=criteria_satisfieds
trainDtDf['Temp Range']=temp_range_levels
trainDtDf.to_csv(TrainDtCsvDir,index='Comp	')

In [15]:
valDataDf=pd.read_csv(ValDtCsvDir0,index_col="ID")
pts_Val=[]
criteria_satisfieds_Val=[]
temp_range_levels_Val=[]
for tbreak,tyield in zip(valDataDf['Tbreak'],valDataDf['YS at Tbreak']):
    pt,criteria_satisfied, temp_range_level=process_Tbreak(tbreak,tyield)
    pts_Val.append(pt)
    criteria_satisfieds_Val.append(criteria_satisfied)
    temp_range_levels_Val.append(temp_range_level)

valDataDf['Assigned Point']=pts_Val
valDataDf['Criteria Satisfied']=criteria_satisfieds_Val
valDataDf['Temp Range']=temp_range_levels_Val
valDataDf.to_csv(ValDtCsvDir0,index='ID')

In [16]:
pts=[]
criteria_satisfieds=[]
temp_range_levels=[]
for tbreak,tyield in zip(trainDtDfFixPP['Tbreak Final'],trainDtDfFixPP['Yield At Tbreak Final']):
    pt,criteria_satisfied, temp_range_level=process_Tbreak(tbreak,tyield)
    pts.append(pt)
    criteria_satisfieds.append(criteria_satisfied)
    temp_range_levels.append(temp_range_level)
trainDtDfFixPP['Assigned Point']=pts
trainDtDfFixPP['Criteria Satisfied']=criteria_satisfieds
trainDtDfFixPP['Temp Range']=temp_range_levels
trainDtDfFixPP.to_csv(TrainDtFxwithPP,index='Comp	')

In [5]:
pts=[]
criteria_satisfieds=[]
temp_range_levels=[]
for tbreak,tyield in zip(TrainExtend['Tbreak'],TrainExtend['YS at Tbreak']):
    pt,criteria_satisfied, temp_range_level=process_Tbreak(tbreak,tyield)
    pts.append(pt)
    criteria_satisfieds.append(criteria_satisfied)
    temp_range_levels.append(temp_range_level)
TrainExtend['Assigned Point']=pts
TrainExtend['Criteria Satisfied']=criteria_satisfieds
TrainExtend['Temp Range']=temp_range_levels
TrainExtend.to_csv(TrainExtendDir,index='ID')